Code to solve the cutting stock problem. 

## Imports, Installations and dependencies

In [1]:
#%pip install pyomo
#%pip install cbc
#%pip install setuptools_scm
#%pip install glpk
%pip install numpy
%pip install matplotlib

  Using cached numpy-1.26.4-cp39-cp39-macosx_11_0_arm64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp39-cp39-macosx_11_0_arm64.whl (14.0 MB)
Note: you may need to restart the kernel to use updated packages.
  Using cached matplotlib-3.9.0-cp39-cp39-macosx_11_0_arm64.whl.metadata (11 kB)
  Using cached contourpy-1.2.1-cp39-cp39-macosx_11_0_arm64.whl.metadata (5.8 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.51.0-cp39-cp39-macosx_10_9_universal2.whl.metadata (159 kB)
  Using cached kiwisolver-1.4.5-cp39-cp39-macosx_11_0_arm64.whl.metadata (6.4 kB)
  Using cached pillow-10.3.0-cp39-cp39-macosx_11_0_arm64.whl.metadata (9.2 kB)
  Using cached pyparsing-3.1.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached importlib_resources-6.4.0-py3-none-any.whl.metadata (3.9 kB)
Using cached matplotlib-3.9.0-cp39-cp39-macosx_11_0_arm64.whl (7.8 MB)
Using cached contourpy-1.2.1-cp39-cp39-macosx_11_0_arm64.whl (244 kB)
Using cached cycler-0.12.1-py3-n

In [2]:
#import pyomo.environ as pyo

import numpy as np
import matplotlib.pyplot as plt
import itertools as it
from itertools import combinations
import sys
import argparse

### Combinatorial Optimization Solution to Cutting Stock

This is brute force combinatorial solver for the cutting stock problem in 1D.

In [ ]:
import itertools as it
from itertools import combinations
import sys
import argparse

# Function to generate the power set of an iterable
def powerset(iterable):
    s = list(iterable) # Convert the iterable to a list
    powerset =  it.chain.from_iterable(combinations(s, r) for r in range(len(s)+1)) # Generate the power set
    return powerset

# Function to sum the lengths of cuts in a given set
def sumcut(cuts, set_elements):
    l = 0 # Initialize the sum of lengths
    
    # Iterate over the cuts in the set
    if len(set_elements[0]) > 0:

        # Sum the lengths of the cuts
        for k in set_elements[0]:
            if len(k) > 0:
                l = l + cuts[k] # Add the length of the cut to the sum
    return l

# Function to build and filter sets of cuts that fit into bars
def buildAndFilterSets(cutsList, barList):
    S = list(powerset(cutsList)) # Generate the power set of cuts
    ret = [] # Initialize the return list
    
    # Iterate over the bars
    for b in barList:
        elements = []

        # Iterate over the cuts
        for c in S:
            tmp = (c, b) # Create a tuple with the cut and the bar
            l = sumcut(cutsList, tmp) # Sum the lengths of the cuts

            # Check if the sum of the lengths is less than or equal to the length of the bar
            if l <= barList[b]:
                elements.append(tmp)
        ret.append(elements)

    U = list(it.product(*ret)) # List of all possible combinations of cuts and bars
    ret = []

    # Iterate over the combinations of cuts and bars
    for cut in U:
        t = list(it.chain.from_iterable([x[0] for x in cut])) # Flatten the list of cuts
         
        # Check if the cuts are unique
        if len(t) == len(set(t)):

            # Check if the cuts are a subset of the power set of cuts
            if len(t) == len(cutsList):
                waste = 0 # Initialize the waste

                # Iterate over the cuts
                for k in cut:
                    l = sumcut(cutsList, k)
                    if l > 0:
                        waste = waste + barList[k[1]] - l
                ret.append([cut, waste])
    return ret

# Main function to parse input arguments and execute the cutting stock problem solution
def CuttingStockSolver(cuts, bars):
    # Initialize dictionaries for bars and cuts
    barre = {}
    k = 0 # counter
    # Iterate over the bars
    for i in bars:
        barre['B' + str(k)] = i # Add the bar to the dictionary
        k = k + 1

    tagli = {} # Initialize the dictionary for the cuts
    k = 0
    # Iterate over the cuts
    for i in cuts:
        tagli['T' + str(k)] = i
        k = k + 1

    print("")
    print("Input Bars(ID: length):", barre)
    print("Input Cuts(ID: length):", tagli)
    print("")
    print("Cuts on bars:")

    TB = buildAndFilterSets(tagli, barre) # Get the best cuts

    # Check if there are cuts
    if len(TB) > 0:
        output = min(TB, key=lambda x: x[1]) # Get the minimum waste
        # Select the best subset of cuts
        ls = [t for t in TB if t[1] <= output[1]]
        # Heuristic: select the best
        u = [[barre[l[1]] - sumcut(tagli, l) for l in k[0]] for k in ls]
        h = [max(k) - min(k) for k in u]
        bestID = h.index(max(h)) # Get the best cut
        # Print the best cut
        for x in ls[bestID][0]:
            print('Bar:', x[1], '(', barre[x[1]], ')', ' Cuts', x[0])
        print('Total waste: ', ls[bestID][1])
        output = ls[bestID]
    else:
        print('No cuts found')
        return 'No cuts found'
    
    return output

## Pyomo Solver for Comparison

In [ ]:
# Function to plot the results of the cutting stock problem
def pyomo_cutting_stock(cuts, bars):
    model = pyo.ConcreteModel()

    # Sets
    model.I = pyo.RangeSet(len(cuts))
    model.J = pyo.RangeSet(len(bars))

    # Parameters
    model.cut_lengths = pyo.Param(model.I, initialize=lambda model, i: cuts[i-1])
    model.bar_lengths = pyo.Param(model.J, initialize=lambda model, j: bars[j-1])

    # Variables
    model.x = pyo.Var(model.I, model.J, domain=pyo.NonNegativeIntegers)

    # Objective: minimize the number of bars used
    model.obj = pyo.Objective(expr=sum(model.x[i, j] for i in model.I for j in model.J), sense=pyo.minimize)

    # Constraints: each cut must be made
    def cuts_constraint_rule(model, i):
        return sum(model.x[i, j] * model.cut_lengths[i] for j in model.J) >= model.cut_lengths[i]
    model.cuts_constraint = pyo.Constraint(model.I, rule=cuts_constraint_rule)

    # Constraints: the sum of cut lengths in each bar cannot exceed the bar length
    def bars_constraint_rule(model, j):
        return sum(model.x[i, j] * model.cut_lengths[i] for i in model.I) <= model.bar_lengths[j]
    model.bars_constraint = pyo.Constraint(model.J, rule=bars_constraint_rule)

    # Solve the model
    solver = pyo.SolverFactory('cbc')
    solver.solve(model, tee=True)

    # Extract results
    results = {j: [] for j in model.J}
    for i in model.I:
        for j in model.J:
            if pyo.value(model.x[i, j]) > 0:
                results[j].append((cuts[i-1], pyo.value(model.x[i, j])))

    return results

In [ ]:
# Examples to test correctness
cuts_example1 = [1.2, 2.5, 3.1, 4.0]
bars_example1 = [5.0, 10.0]
CuttingStockSolver(cuts_example1, bars_example1)

In [ ]:
# Function to compare results
def compare_solvers(cuts, bars):
    # Your solver
    user_solution = CuttingStockSolver(cuts, bars)
    
    # Pyomo solver
    pyomo_solution = pyomo_cutting_stock(cuts, bars)
    
    print("Your Solution:")
    print(your_solution)
    
    print("\nPyomo Solution:")
    print(pyomo_solution)

# Examples to test correctness
cuts_example1 = [1.2, 2.5, 3.1, 4.0]
bars_example1 = [5.0, 10.0]
compare_solvers(cuts_example1, bars_example1)
